#**Roberta For Fake News**

> Este trabajo se enfoca en la detección de noticias falsas en español, un desafío crítico en la era digital. Para abordar este problema, se utiliza el modelo RoBERTa junto con técnicas de procesamiento de lenguaje natural. El proceso implica la recopilación y curación de un conjunto de datos en español que abarque noticias auténticas y falsas, considerando diversos temas y estilos de escritura. Una vez que se cuenta con el conjunto de datos adecuado, se configuran hiper parámetros específicos del idioma y se lleva a cabo la tokenización adecuada para adaptar el modelo RoBERTa al español. Luego, se procede a entrenar el modelo utilizando técnicas de aprendizaje profundo, como el ajuste fino, con el objetivo de desarrollar un sistema de detección de noticias falsas eficaz en español. El proceso de entrenamiento y evaluación involucra la separación de datos en conjuntos de entrenamiento, validación y prueba para ajustar los hiper parámetros y evitar el sobreajuste. Una vez que se ha logrado un rendimiento satisfactorio, el modelo entrenado se puede implementar en aplicaciones o sistemas, contribuyendo de esta manera a la mitigación de la difusión de información falsa y engañosa en el ámbito de habla hispana.



##**Requerimientos**


> Paquetes para Mapeo y ETL

*   HuggingFace: transformers, datasets
*   Kaggle:




---




In [1]:
!pip install accelerate==0.20.1 transformers[torch] datasets #para el trainning /librerías específicas #TODO/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 22.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers[torch] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 120.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 125.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 129.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#Vamos a instalar todos los módulos

!pip install kaggle #Para mapeo de Kaggle

!pip install transformers datasets #Para mapeo en hugging face

!pip install Pyspark #Para el ETL

!huggingface-cli login


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=9bcd28fb6a604bb29c4a4cd9c0fe16583400f647a2d0d36bf0be3a3a6e483083
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built Pyspark

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log

##**Mapeo de datasets y ETL**

---



###**Mapeo de datasets en HuggingFace y Kaggle**

####**Kaggle**
Vamos a mapear datasets de noticias falses en español usando la API de Kaggle.

#####**Mapeo de Datos**

In [3]:
#Primero vamos a importar la función "files" del módulo  google.colab

from google.colab import files

In [4]:
#Acá subimos la llave .json de nuestra cuenta de Kaggle

files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [5]:
#Buscamos los datasets relacionados a "news"

!kaggle datasets list -s "news"

ref                                                  title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rmisra/news-category-dataset                         News Category Dataset                          27MB  2022-09-24 08:07:21          57022        691  1.0              
hacker-news/hacker-news                              Hacker News                                      0B  2019-02-12 00:34:51              0        523  0.7058824        
therohk/million-headlines                            A Million News Headlines                       21MB  2022-06-11 14:08:07          39493        635  1.0              
gpreda/bbc-news                                      BBC News                                        2MB  2023-11-20 00:27:56           2063     

In [6]:
#Miramos los documentos que tiene el dataset
!kaggle datasets files javieroterovizoso/spanish-political-fake-news

name                 size  creationDate         
-------------------  ----  -------------------  
D11000_test.csv       4MB  2023-11-03 16:28:06  
D21000_train.csv      7MB  2023-11-03 16:28:06  
ReadMe.txt           334B  2023-11-03 16:28:06  
D46000_train.csv     15MB  2023-11-03 16:28:06  
D5500_test.csv        2MB  2023-11-03 16:28:06  
D57000_complete.csv  19MB  2023-11-03 16:28:06  


#####**Descarga de datasets de Kaggle**
El dataset que descargaremos de Kaggle es **"javieroterovizoso/spanish-political-fake-news"**
que tiene noticias etiquetadas en español. Y descargaremos la versión compacta:

*   D57000_complete.csv  19MB

In [7]:
!kaggle datasets download -d javieroterovizoso/spanish-political-fake-news -f D57000_complete.csv

 71% 5.00M/7.02M [00:00<00:00, 32.1MB/s]
100% 7.02M/7.02M [00:00<00:00, 42.7MB/s]


#####**Descomprimimos los datasets de Kaggle**

In [8]:
import zipfile

# Nombre del archivo ZIP descargado
zip_filename = "D57000_complete.csv.zip"

# Directorio de destino para descomprimir
extract_dir = "/content/extracted_data"

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
  zip_ref.extractall(extract_dir)


In [9]:
#Ahora descargamos el otro dataset

!kaggle datasets files arseniitretiakov/noticias-falsas-en-espaol

name                size  creationDate         
-----------------  -----  -------------------  
fakes1000.csv      484KB  2020-08-18 12:53:00  
train.csv          474KB  2020-08-18 12:53:00  
onlytrue1000.csv   238KB  2020-08-18 12:53:00  
test.csv           112KB  2020-08-18 12:53:00  
onlyfakes1000.csv  234KB  2020-08-18 12:53:00  


In [10]:
!kaggle datasets download -d arseniitretiakov/noticias-falsas-en-espaol -f fakes1000.csv

  0% 0.00/484k [00:00<?, ?B/s]
100% 484k/484k [00:00<00:00, 125MB/s]


####**HuggingFace**

#####**Descargamos el dataset**

In [11]:
from datasets import load_dataset

dataset_train = load_dataset("sayalaruano/FakeNewsCorpusSpanish", split="train", column_names=['Id', 'Category', 'Topics', 'Source', 'Headline', 'Text', 'Link'])
print(f"Train dataset size: {len(dataset_train)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Train dataset size: 677


In [12]:
dataset_test = load_dataset("sayalaruano/FakeNewsCorpusSpanish", split="test", column_names=['Id', 'Category', 'Topics', 'Source', 'Headline', 'Text', 'Link'])
print(f"Train dataset size: {len(dataset_test)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Train dataset size: 573


In [13]:
df_p_HF_train = dataset_train.to_pandas()
df_p_HF_test = dataset_test.to_pandas()

###**ETL de los datasets mapeados**

In [14]:
#Transformar esos csv a Dataframes con Pyspark

from pyspark.sql import SparkSession

# Inicializa la sesión de Spark
spark = SparkSession.builder.appName("ALLDataset").getOrCreate()

# Carga un archivo CSV en un DataFrame de PySpark
df_s_KG_D57000_complete = spark.read.csv("/content/extracted_data/D57000_complete.csv", header=True, inferSchema=True, sep=';')

df_s_KG_D57000_complete.show()

+---+-----+--------------------+--------------------+----------+
| ID|Label|              Titulo|         Descripcion|     Fecha|
+---+-----+--------------------+--------------------+----------+
| ID|    1|Moreno intenta ap...|El presidente abr...|19/04/2022|
| ID|    1|La Abogacía del E...|En un escrito, la...|17/09/2021|
| ID|    0|Las promesas incu...|Este lunes y mart...|12/09/2022|
| ID|    1|Sánchez defiende ...|Resulta evidente ...|07/02/2023|
| ID|    1|Ian Gibson cierra...|El hispanista, qu...|12/04/2023|
| ID|    1|ERC y Junts se at...|Empieza una inten...|19/09/2022|
| ID|    1|El PP se rinde al...|Núñez Feijóo guar...|13/01/2023|
| ID|    1|El PSOE denuncia ...|Los socialistas h...|31/03/2023|
| ID|    1|Escrivá niega que...|El ministro de In...|08/08/2022|
| ID|    1|IU propone una de...|Izquierda Unida d...|26/11/2022|
| ID|    1|Ayuso rescata a V...|La presidenta mad...|25/04/2021|
| ID|    0|Los 33 días que f...|El 16 de febrero ...|21/03/2022|
| ID|    0|El Nueva Canar

In [15]:
#Veemos el esquema del dataframe

df_s_KG_D57000_complete.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Label: integer (nullable = true)
 |-- Titulo: string (nullable = true)
 |-- Descripcion: string (nullable = true)
 |-- Fecha: string (nullable = true)



In [16]:
from pyspark.sql.functions import col
def Registros_Nulos_SQL(data_frame):

    data_frame.createOrReplaceTempView("tbl_data_frame")

    resultado = spark.sql("SELECT COUNT(*) AS ValoresNulos FROM tbl_data_frame WHERE Descripcion IS NULL")

    resultado.show()

In [17]:
Registros_Nulos_SQL(df_s_KG_D57000_complete)

+------------+
|ValoresNulos|
+------------+
|           0|
+------------+



In [18]:
#Conservamos campos que nos interesen
df_s_KG_D57000_complete = df_s_KG_D57000_complete.drop(*('ID', 'Titulo', 'Fecha'))
df_s_KG_D57000_complete.show()

+-----+--------------------+
|Label|         Descripcion|
+-----+--------------------+
|    1|El presidente abr...|
|    1|En un escrito, la...|
|    0|Este lunes y mart...|
|    1|Resulta evidente ...|
|    1|El hispanista, qu...|
|    1|Empieza una inten...|
|    1|Núñez Feijóo guar...|
|    1|Los socialistas h...|
|    1|El ministro de In...|
|    1|Izquierda Unida d...|
|    1|La presidenta mad...|
|    0|El 16 de febrero ...|
|    0|La oposición pide...|
|    0|El Congreso acuer...|
|    1|Los afectados ago...|
|    1|Yolanda Díaz se p...|
|    1|Creo que no vamos...|
|    1|Un concejal del p...|
|    1|Los beneficiarios...|
|    1|El presidente del...|
+-----+--------------------+
only showing top 20 rows



In [19]:
#Ahora con el dataset de fakes1000

df_s_KG_fakes1000 = spark.read.csv("/content/fakes1000.csv", header=True, inferSchema=True, sep=',')

df_s_KG_fakes1000.show()

+-----+--------------------+
|class|                Text|
+-----+--------------------+
| TRUE| Algunas de las v...|
| TRUE| Después de casi ...|
| TRUE| Dos periodistas ...|
| TRUE| El Cuerpo Nacion...|
| TRUE| El desfile de la...|
| TRUE| El miedo a la pé...|
| TRUE| El Ministerio de...|
| TRUE| El nuevo plan mi...|
| TRUE|" El secretario g...|
|FALSE| El suceso ha ten...|
| TRUE| Ernest Maragall,...|
| TRUE| España es líder ...|
|FALSE| Estamos en la se...|
|FALSE| Estudios científ...|
| TRUE|" Existe un lugar...|
| TRUE| Gente de todas l...|
|FALSE| Ha sucedido en l...|
|FALSE|" La fiesta en Se...|
| TRUE| La juez de Instr...|
| TRUE| La mujer pakista...|
+-----+--------------------+
only showing top 20 rows



In [20]:
df_s_KG_fakes1000 = df_s_KG_fakes1000.withColumnRenamed("class", "Label")

df_s_KG_fakes1000 = df_s_KG_fakes1000.withColumnRenamed("Text", "Descripcion")

# Mostrar el DataFrame con el nuevo nombre de columna
df_s_KG_fakes1000.show()

+-----+--------------------+
|Label|         Descripcion|
+-----+--------------------+
| TRUE| Algunas de las v...|
| TRUE| Después de casi ...|
| TRUE| Dos periodistas ...|
| TRUE| El Cuerpo Nacion...|
| TRUE| El desfile de la...|
| TRUE| El miedo a la pé...|
| TRUE| El Ministerio de...|
| TRUE| El nuevo plan mi...|
| TRUE|" El secretario g...|
|FALSE| El suceso ha ten...|
| TRUE| Ernest Maragall,...|
| TRUE| España es líder ...|
|FALSE| Estamos en la se...|
|FALSE| Estudios científ...|
| TRUE|" Existe un lugar...|
| TRUE| Gente de todas l...|
|FALSE| Ha sucedido en l...|
|FALSE|" La fiesta en Se...|
| TRUE| La juez de Instr...|
| TRUE| La mujer pakista...|
+-----+--------------------+
only showing top 20 rows



In [21]:
df_s_KG_fakes1000.printSchema()

root
 |-- Label: string (nullable = true)
 |-- Descripcion: string (nullable = true)



In [22]:
Registros_Nulos_SQL(df_s_KG_fakes1000)

+------------+
|ValoresNulos|
+------------+
|           1|
+------------+



In [23]:
df_s_KG_fakes1000 = df_s_KG_fakes1000.dropna()

In [24]:
Registros_Nulos_SQL(df_s_KG_fakes1000)

+------------+
|ValoresNulos|
+------------+
|           0|
+------------+



In [25]:
from pyspark.sql.functions import when
df_s_KG_fakes1000 = df_s_KG_fakes1000.withColumn("Label", when(col("Label") == "TRUE", 1).otherwise(0))

In [26]:
df_s_KG_fakes1000.groupby('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|    1| 1000|
|    0| 1001|
+-----+-----+



In [27]:
#Cambiar tipo de dato de columna
df_s_KG_fakes1000 = df_s_KG_fakes1000.withColumn("Label", col("Label").cast("int"))

In [28]:
df_s_KG_fakes1000.head()

Row(Label=1, Descripcion=' Algunas de las voces extremistas más conocidas de EE.UU., cuentas asociadas con al movimiento de la conocida como alt-right o ligadas a grupos antisemitas o xenófobos , han sido eliminadas de un plumazo de la redes sociales Facebook e Instagram. Se trat')

In [ ]:
#Label '1' = True, Label '0'= False

In [ ]:
#Unir el dataset de entrenamiento y el test, algunos si vienen etiquetados apropiadamente, asi tenemos más datos para entrenar

###**=======================MEJORAR=======================**
Vamos a eliminar registros para que el dataframe tenga igual número de registros por etiqueta, esto es para usar la métrica "accuracy" de datsets de HuggingFace. Hasta la fecha no encontré la métrica adecuada para datos asimétricos en clasificación de texto para que el entrenamiento se ejecute.

En la documentación de datasets de Hugging Face mencionan la definición de métricas para datasets asimétricos pero no se ejecutan correctamente en el colab.

Esto tendrá repercusiones en la parte del **Entrenamiento**


In [ ]:
df_label_1 = df_K1_train.filter(df_K1_train["Label"] == '1')
df_label_0 = df_K1_train.filter(df_K1_train["Label"] == '0')

In [ ]:
df_label_1 = df_label_1.limit(9060)

In [ ]:
df_label_1.groupby('Label').count().show()
df_label_0.groupby('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|    1| 9060|
+-----+-----+

+-----+-----+
|Label|count|
+-----+-----+
|    0| 9060|
+-----+-----+



In [ ]:
df_label_1 = df_label_1.unionAll(df_label_0)

In [ ]:
df_label_1.groupby('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|    1| 9060|
|    0| 9060|
+-----+-----+



In [ ]:
df_K1_train = df_label_1

In [ ]:
df_K1_train.groupby('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|    1| 9060|
|    0| 9060|
+-----+-----+



In [ ]:
#Mover este dataset en el fichero "DataSetFormateado"

In [36]:
#Concactenar los dataframes de HuggingFace

import pandas as pd

df_p_HF_tt = concatenado = pd.concat([df_p_HF_train, df_p_HF_test], axis=1)

In [40]:
df_p_HF_train.head()

,Id,Category,Topics,Source,Headline,Text,Link
0,Id,Category,Topic,Source,Headline,Text,Link
1,1,Fake,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...
2,2,Fake,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...
3,3,Fake,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...
4,4,True,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...


In [ ]:
df_s_HF_tt =

In [41]:
df_p_HF_tt.head()

,Id,Category,Topics,Source,Headline,Text,Link,Id,Category,Topics,Source,Headline,Text,Link
0,Id,Category,Topic,Source,Headline,Text,Link,ID,CATEGORY,TOPICS,SOURCE,HEADLINE,TEXT,LINK
1,1,Fake,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...,1,1,Covid-19,El Economista,Covid-19: mentiras que matan,El control de la Covid-19 no es sólo un tema d...,https://www.eleconomista.com.mx/opinion/Covid-...
2,2,Fake,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...,2,0,Política,El matinal,El Gobierno podrá acceder a las IPs de los móv...,El Gobierno de Pedro Sánchez y Pablo Iglesias ...,https://www.elmatinal.com/espana-ultima-hora/e...
3,3,Fake,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...,3,1,Política,El País,La comunidad musulmana catalana denuncia a Vox...,Las tres federaciones que agrupan al 90% de la...,https://elpais.com/espana/elecciones-catalanas...
4,4,True,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...,4,0,Política,AFPFactual,None,Se han dado a conocer los datos electorales pr...,https://perma.cc/GYE6-SPMB


In [ ]:
#datasetFormateado

df_s_KG_fakes1000


##**Entrenar**


---



###Tokenizar

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "BSC-TeMU/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.vocab_size

50262

In [ ]:
text = "¡Hola! Esto es un ejemplo de texto, adiós."
tokenized_text = tokenizer.encode(text)

for token in tokenized_text:
    print(token, tokenizer.decode([token]))

0 <s>
1465 ¡
5700 Hola
55 !
22591  Esto
347  es
355  un
1915  ejemplo
313  de
3487  texto
66 ,
15989  adiós
68 .
2 </s>


In [ ]:
encoded_text = tokenizer(text, return_tensors="pt")
encoded_text

{'input_ids': tensor([[    0,  1465,  5700,    55, 22591,   347,   355,  1915,   313,  3487,
            66, 15989,    68,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
def tokenize_descripcion(examples):
    return tokenizer(examples["Descripcion"], truncation=True)

In [ ]:
from datasets import Dataset

ds_hf = Dataset.from_spark(df_K1_train)

columns = ds_hf.column_names
columns.remove("Label")
encoded_dataset = ds_hf.map(tokenize_descripcion, batched=True, remove_columns=columns)
encoded_dataset

Map:   0%|          | 0/18120 [00:00<?, ? examples/s]

Dataset({
    features: ['Label', 'input_ids', 'attention_mask'],
    num_rows: 18120
})

In [ ]:
ds_hf

Dataset({
    features: ['Label', 'Descripcion'],
    num_rows: 18120
})

In [ ]:
encoded_dataset[0]

{'Label': 1,
 'input_ids': [0,
  496,
  2385,
  1840,
  313,
  27296,
  509,
  332,
  9323,
  38636,
  320,
  22129,
  334,
  3666,
  394,
  2492,
  341,
  2405,
  355,
  1658,
  350,
  395,
  410,
  1504,
  66,
  49303,
  361,
  38563,
  350,
  344,
  1856,
  369,
  7972,
  313,
  5239,
  14631,
  383,
  19676,
  11237,
  1963,
  139,
  68,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

###Cargar Modelo Preentrenado

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
outputs = model(**encoded_text)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[0.0195, 0.0782]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

###Definir métricas

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")
metric

<ipython-input-41-6ecf60b6e792>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

###Fine tuning

In [ ]:
!pip install accelerate -U


In [ ]:
import transformers
import accelerate

print("Versión de Transformers:", transformers.__version__)
print("Versión de Accelerate:", accelerate.__version__)

Versión de Transformers: 4.35.2
Versión de Accelerate: 0.24.1


In [ ]:
from transformers import TrainingArguments

print("Versión de Transformers:", transformers.__version__)
print("Versión de Accelerate:", accelerate.__version__)


model_name = model_checkpoint.split("/")[-1]

batch_size = 16
num_train_epochs=2
num_train_samples = 20000
train_dataset = encoded_dataset.shuffle(seed=23).select(range(num_train_samples))
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-amazon_reviews_multi"
)

Versión de Transformers: 4.35.2
Versión de Accelerate: 0.24.1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1716: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case MyBad2K/roberta-base-bne-finetuned-amazon_reviews_multi).
  warnings.warn(


In [ ]:
df_K1_test = spark.read.csv("/content/extracted_data/D5500_test.csv", header=True, inferSchema=True, sep=';')

df_K1_test.show()

+---+-----+--------------------+--------------------+----------+
| ID|Label|              Titulo|         Descripcion|     Fecha|
+---+-----+--------------------+--------------------+----------+
| ID|    1|Las primarias par...|Unos 7.350 de mil...|23/09/2018|
| ID|    1|Un empresario vin...|Pedro Veiga, pare...|22/12/2022|
| ID|    0|Santamaría se sum...|La izquierda se e...|21/07/2018|
| ID|    0|El presidente del...|El secretario gen...|08/10/2019|
| ID|    0|El Iniciativa ver...|Los socialistas v...|02/07/2019|
| ID|    1|¿Por qué apenas h...|El escaso interés...|20/05/2023|
| ID|    1|El Congreso aprue...|La celebración de...|15/12/2022|
| ID|    1|Un juzgado abre u...|El Juzgado de Ins...|14/09/2021|
| ID|    1|Empleo nombra alt...|Fuentes del Gobie...|21/11/2018|
| ID|    0|El Iniciativa ver...|Pide que llame a ...|21/05/2019|
| ID|    0|Belarra pide a PS...|Todo el mundo tie...|06/07/2020|
| ID|    1|Jácome asegura qu...|El alcalde de Our...|28/12/2021|
| ID|    1|Los trabajador

In [ ]:
#
df_K1_test.groupby('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|    1| 3191|
|    0| 2485|
+-----+-----+



In [ ]:
df_K1_test = df_K1_test.drop(*('ID', 'Titulo', 'Fecha'))

In [ ]:
from datasets import Dataset

ds_hf_t = Dataset.from_spark(df_K1_test)

columns = ds_hf_t.column_names
columns.remove("Label")
encoded_dataset_t = ds_hf_t.map(tokenize_descripcion, batched=True, remove_columns=columns)
encoded_dataset_t

Map:   0%|          | 0/5676 [00:00<?, ? examples/s]

Dataset({
    features: ['Label', 'input_ids', 'attention_mask'],
    num_rows: 5676
})

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset_t,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

ValueError: ignored

##**Evaluar**

---



##**Desplegar**

---

